# Exercise 3

## Import packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

from tqdm import trange, tqdm

## Task 1
(3 points)

Implement the training loop for one training epoch.
An epoch trains on the whole training dataset once.

In [2]:
def train(model, use_cuda, train_loader, optimizer, epoch, log_interval):
    """
    Train one epoch
    
    model -- the neural network
    use_cuda -- true if GPU should be used
    train_loader -- data loader
    optimizer -- network optimizer
    epoch -- number of current epoch
    log_interval -- number of training steps between logs
    """

    criterion = F.nll_loss # negative log likelihood loss here

    # Use GPU if available
    device = torch.device("cuda" if use_cuda else "cpu")
    model.to(device)

    # TODO: set the model to train mode
    model.train()
    # TODO: enumerate over the dataloader to get mini batches
    #       of images and ground truth labels
    # HINT: the builtin python function enumerate() also gives you indices
    for batch_idx,item in enumerate(train_loader):
        # get the images and the truth labels
        images, truth_labels = item
        images, truth_labels = images.to(device), truth_labels.to(device)

        # TODO: set the optimizers gradients to zero
        optimizer.zero_grad()
        # TODO: run the network
        outputs = model(images)
        # TODO: compute negative log likelihood loss
        loss = criterion(outputs,truth_labels)
        # TODO: do backpropagation
        loss.backward()
        # TODO: optimize
        optimizer.step() # update the weights

        # TODO: print current loss for every nth ("log_interval"th) iteration
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(images)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

We already implemented the validation function for you (this is essentially validate() from the last exercise)

In [3]:
def validate(model, use_cuda, test_loader):
    """
    Compute test metrics
    
    model -- the neural network
    use_cuda -- true if GPU should be used
    test_loader -- data loader
    """
    # create a 10x10 grid of subplots
    _, axis = plt.subplots(10, 10)
    
    # set model to evaluation mode
    model.eval()
    test_loss = 0
    correct = 0
    plotted = 0
    
    # disable gradients globally
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            # for each batch
            if use_cuda:
                # transfer to GPU
                data = data.cuda()
                target = target.cuda()
            
            # run network and compute metrics
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            
            img_correct = pred.eq(target.view_as(pred))
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            # plot the first 100 images
            img_idx = 0
            data = data.cpu().numpy()
            
            #while plotted < 100 and img_idx < data.shape[0]:
            while plotted < 10 and img_idx < data.shape[0]:# i reduced the number of plots
                # compute position of ith image in the grid
                y = plotted % 10
                x = plotted // 10
                
                # convert image tensor to numpy array and normalize to [0, 1]
                img = data[img_idx, 0]
                img = (img - np.min(img)) / (np.max(img) - np.min(img))
                
                # make wrongly predicted images red
                img = np.stack([img] * 3, 2)
                if img_correct[img_idx] == 0:
                    img[:, :, 1:] = 0.0
                
                # disable axis and show image
                axis[y][x].axis('off')
                axis[y][x].imshow(img)
                
                # show the predicted class next to each image
                axis[y][x].text(30, 25, pred[img_idx].item())
                
                plotted += 1
                img_idx += 1
            
    test_loss /= len(test_loader.dataset)

    # show results
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    plt.show()

## Task 2
(4 points)

Implement a five-layer fully connected neural network.
The dimensions (without batch size) should change like this: 784->200->100->60->30->10
Use log softmax to compute the class predictions.

Run the code at the end of the notebook to train and validate your implementation.

### Task 2.1
* sigmoid non-linear activation function
* note that the last layer does not need an activation function!

### Task 2.2
* add a new class "FCNet2"
* replace sigmoid with ReLU

### Task 2.3
* add a new class "FCNet2"
* add batch normalization to the first and third layers (note the difference between 1D/2D/3D versions)


**NOTE:** The perfomance should improve slightly with each step. However, due to the random weight initialization applied by PyTorch, your results may vary a bit between trainings.

In [4]:
class FCNet1(nn.Module):
    """
    Fully Connected Neural Network
    
    Five fully connected layers with sigmoid non-linearity
    
    Dimensions
    784->200->100->60->30->10
    """
    def __init__(self, layer_size_list=[784, 200, 100, 60, 30, 10]): # add the dropout probability here as well
        super(FCNet1, self).__init__()
        self.layer_size_list = layer_size_list
        # TODO: initialize network layers
        # HINT: take a look at "torch.nn" (imported as "nn")
        self.linear_layers = nn.ModuleList()
        for i in range(len(self.layer_size_list) - 1):
            self.linear_layers.append(
                nn.Linear(self.layer_size_list[i], self.layer_size_list[i + 1])
            )
    def forward(self, x):
        # TODO: reshape batch of images to batch of 1D vectors
        x = x.view(x.size(0), -1)# This would reshape input from (batch_size, 1, 28, 28) to (batch_size, 784)
        
        # TODO: run network layers
        for layer in self.linear_layers[:-1]:
            x = torch.sigmoid(layer(x))
        # Apply the final layer (no activation)
        x = self.linear_layers[-1](x)

        # TODO: compute log softmax over the output
        output = F.log_softmax(x, dim=1)

        return output

In [5]:
class FCNet2(nn.Module):
    def __init__(self, layer_size_list=[784, 200, 100, 60, 30, 10]):# add dropout later
        super(FCNet2, self).__init__()

        self.layer_size_list = layer_size_list
        
        self.linear_layers = nn.ModuleList() # this is list of linear layer
        for i in range(len(self.layer_size_list) - 1):
            self.linear_layers.append(
                nn.Linear(self.layer_size_list[i], self.layer_size_list[i + 1])
            )
    def forward(self, x):
        # TODO: reshape batch of images to batch of 1D vectors
        x = x.view(x.size(0), -1)# This would reshape input from (batch_size, 1, 28, 28) to (batch_size, 784)
        
        # TODO: run network layers
        for layer in self.linear_layers[:-1]:
            x = F.relu(layer(x))
        # Apply the final layer (no activation)
        x = self.linear_layers[-1](x)

        # TODO: compute log softmax over the output
        output = F.log_softmax(x, dim=1)

        return output

In [6]:
class FCNet3(nn.Module):
    def __init__(self, layer_size_list=[784, 200, 100, 60, 30, 10]):

        super(FCNet3, self).__init__()

        self.layer_size_list = layer_size_list

        self.linear_layers = nn.ModuleList() # this is list of linear layer
        self.batchnorm_layers = nn.ModuleDict() # list of BN layer
        
        for i in range(len(self.layer_size_list) - 1):
            self.linear_layers.append(
                nn.Linear(self.layer_size_list[i], self.layer_size_list[i + 1])
            )
            if i==0 or i==2:
                self.batchnorm_layers[str(i)]=nn.BatchNorm1d(self.layer_size_list[i + 1])

    def forward(self, x):
        # TODO: reshape batch of images to batch of 1D vectors
        x = x.view(x.size(0), -1)# This would reshape input from (batch_size, 1, 28, 28) to (batch_size, 784)

        # TODO: run network layers
        for i in range(len(self.linear_layers)-1):
            x=self.linear_layers[i](x)
            # apply BN layer to 1st and 3rd layer
            if i==0 or i==2:
                x=self.batchnorm_layers[str(i)](x)
            # apply activation function
            x = F.relu(x)
        # Apply the final layer (no activation)
        x = self.linear_layers[-1](x)

        # TODO: compute log softmax over the output
        output = F.log_softmax(x, dim=1)

        return output

## Task 3
(3 points)

Implement a convolutional neural network, consisting of two convolutional and two fully connected layers.
This time, the dimensions (without batch size) should change like this: 1x28x28->32x26x26->64x12x12->128->10

### Task 3.1
* two convolutional layers (kernel size 3)
* two fully-connected layers
* ReLU activation function

### Task 3.2
* add batch normalization to first convolutional and first fully connected layer

### Task 3.3
* use max pooling instead of stride to reduce the dimensions to 64x12x12

Ans: 

Note these formulae:

1. For a simple kernel and stride based donwsampling:
* O = ((W - K + 2P) / S) + 1

Where:

* O is the output size
* W is the input size (width or height)
* K is the kernel size
* P is the padding
* S is the stride

2. For maxpooling based downsampling:
* O= $\left \lfloor\frac{(W + 2P - Dialation \times (k-1)-1 )}{S} +1 \right \rfloor$

Dilation is 1 for maxpooling 

In [7]:
class ConvNet1(nn.Module):
    """
    Convolutional Neural Network
    
    Two convolutional layers and two fully connected layers
    
    Dimensions:
    1x28x28->32x26x26->64x12x12->128->10
    """
    def __init__(self):
        super(ConvNet1, self).__init__()
        
        # TODO: initialize network layers
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)  # 28x28 -> 26x26
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2) # 26x26 -> 12x12

        # Flattened size after conv2: 64 * 12 * 12 = 9216
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 10)
        

    def forward(self, x):
        # Conv layers with ReLU
        x = F.relu(self.conv1(x))  # 1x28x28 -> 32x26x26
        x = F.relu(self.conv2(x))  # 32x26x26 -> 64x24x24

        # Flatten
        x = x.view(x.size(0), -1)  # batch x 9216

        # Fully connected layers with ReLU
        x = F.relu(self.fc1(x))    # 9216 -> 128
        x = self.fc2(x)            # 128 -> 10

        # Log softmax for class probabilities
        output = F.log_softmax(x, dim=1)
        
        return output

In [8]:
class ConvNet2(nn.Module):
    """
    Convolutional Neural Network
    
    Two convolutional layers and two fully connected layers
    Batch normalisation after 1CNN and 1FC layer
    
    Dimensions:
    1x28x28->32x26x26->64x12x12->128->10
    """
    def __init__(self):
        super(ConvNet2, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)   # 28x28 -> 26x26
        self.bn1 = nn.BatchNorm2d(32)  # BatchNorm after conv1

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2)  # 26x26 -> 12 x 12

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.bn_fc1 = nn.BatchNorm1d(128)  # BatchNorm after first FC
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Conv layer 1 + BN + ReLU
        x = F.relu(self.bn1(self.conv1(x)))   # 1x28x28 -> 32x26x26
        # Conv layer 2 + ReLU
        x = F.relu(self.conv2(x))             # 32x26x26 -> 64x12x12
        # Flatten
        x = x.view(x.size(0), -1)             # batch x 9216
        # FC layer 1 + BN + ReLU
        x = F.relu(self.bn_fc1(self.fc1(x)))  # 9216 -> 128
        # FC layer 2
        x = self.fc2(x)                       # 128 -> 10
        # LogSoftmax
        output = F.log_softmax(x, dim=1)

        return output


If we choose k=4,P=0, Dilation=1, and stride =3 for input image of W=26, then :

O=((26+0-4)/2)+1 => O = 11 + 1 = 12!

In [14]:
class ConvNet3(nn.Module):
    def __init__(self):
        super(ConvNet3, self).__init__()
        
        # TODO: initialize network layers
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3)  # 28x28 -> 26x26
        self.maxpool1 = nn.MaxPool2d(kernel_size=4, stride=2) # 26x26 -> 12x12

        # Flattened size after conv2: 64 * 12 * 12 = 9216
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 10)
        

    def forward(self, x):
        # Conv layers with ReLU
        x = F.relu(self.conv1(x))  # 1x28x28 -> 32x26x26
        x = F.relu(self.maxpool1(x))  # 32x26x26 -> 64x24x24

        # Flatten
        x = x.view(x.size(0), -1)  # batch x 9216

        # Fully connected layers with ReLU
        x = F.relu(self.fc1(x))    # 9216 -> 128
        x = self.fc2(x)            # 128 -> 10

        # Log softmax for class probabilities
        output = F.log_softmax(x, dim=1)
        
        return output

### Hyperparameters and DataLoader

In [10]:
# hyper parameters
batch_size = 64
test_batch_size = 1000
epochs = 2#10
lr = 1.0
gamma = 0.7
log_interval = 100

# use GPU if available
use_cuda = False#torch.cuda.is_available()
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# initialize data loaders
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])), batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, 
    transform=transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

### Fully connected networks

In the code below, IDK why when I keep `validate()` function, the kernel keeps on crashing on my laptop, thus meanwhile I have removed the validation function in the code below. (note that this code was in already present in the code, in the cell above, so we cannot simply comment `validate` function in the final solution.).


I ran script in google colab and it worked quite well. try it in your systems as well. if everything works well, then change the number of epochs to 10 in the cell above.

In [ ]:
model = FCNet1()
if use_cuda:
    model = model.cuda()

# initialize optimizer and scheduler
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

for epoch in trange(1, epochs + 1, desc="Epochs"):
    # train one epoch
    train(model, use_cuda, train_loader, optimizer, epoch, log_interval)
    
    # run on test dataset
    #validate(model, use_cuda, test_loader)
    # I think there is some problem in the validate model
    scheduler.step()
    
    #torch.save(model.state_dict(), "models/mnist/checkpoint.pt")

In [ ]:
model = FCNet2()
if use_cuda:
    model = model.cuda()

# initialize optimizer and scheduler
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

for epoch in range(1, epochs + 1):
    # train one epoch
    train(model, use_cuda, train_loader, optimizer, epoch, log_interval)
    
    # run on test dataset
    # validate(model, use_cuda, test_loader) 
    scheduler.step()
    
    
    #torch.save(model.state_dict(), "models/mnist/checkpoint.pt")

In [ ]:
model = FCNet3()
if use_cuda:
    model = model.cuda()

# initialize optimizer and scheduler
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

for epoch in range(1, epochs + 1):
    # train one epoch
    train(model, use_cuda, train_loader, optimizer, epoch, log_interval)
    
    # run on test dataset
    # validate(model, use_cuda, test_loader)
    scheduler.step()
    
    
    #torch.save(model.state_dict(), "models/mnist/checkpoint.pt")

### Running CNN networks

In [ ]:
model = ConvNet1()
if use_cuda:
    model = model.cuda()

# initialize optimizer and scheduler
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

for epoch in trange(1, epochs + 1, desc="Epochs"):
    # train one epoch
    train(model, use_cuda, train_loader, optimizer, epoch, log_interval)
    
    # run on test dataset
    #validate(model, use_cuda, test_loader)
    # I think there is some problem in the validate model
    scheduler.step()
    
    #torch.save(model.state_dict(), "models/mnist/checkpoint.pt")

In [ ]:
model = ConvNet2()
if use_cuda:
    model = model.cuda()

# initialize optimizer and scheduler
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

for epoch in trange(1, epochs + 1, desc="Epochs"):
    # train one epoch
    train(model, use_cuda, train_loader, optimizer, epoch, log_interval)
    
    # run on test dataset
    #validate(model, use_cuda, test_loader)
    # I think there is some problem in the validate model
    scheduler.step()
    
    #torch.save(model.state_dict(), "models/mnist/checkpoint.pt")

In [ ]:
model = ConvNet3()
if use_cuda:
    model = model.cuda()

# initialize optimizer and scheduler
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

for epoch in trange(1, epochs + 1, desc="Epochs"):
    # train one epoch
    train(model, use_cuda, train_loader, optimizer, epoch, log_interval)
    
    # run on test dataset
    #validate(model, use_cuda, test_loader)
    # I think there is some problem in the validate model
    scheduler.step()
    
    #torch.save(model.state_dict(), "models/mnist/checkpoint.pt")